# Exercício 1 - Enunciado

Considere-se de novo o algoritmo estendido de Euclides apresentado no TP2  mas usando o tipo dos inteiros e um parâmetro $N>0$
```
    INPUT  a, b : Int
    assume  a > 0 and b > 0 and a < N and b < N
    r, r', s, s', t, t' = a, b, 1, 0, 0, 1
    while r' != 0
      q = r div r'
      r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 
    OUTPUT r, s, t
```

Este exercício é dirigido às provas de segurança do algoritmo acima.

1. Construa um FOTS $\Sigma\,\equiv\,\langle\mathsf{X}, \mathsf{I},\mathsf{T}\rangle\,$ usando este modelo nos inteiros. 
2. Considere como propriedade de segurança
                        `safety = (r > 0) and (r < N) and (r = a*s + b*t)`
    Prove usando $k$-indução que esta propriedade se verifica em qualquer traço do FOTS
3. Prove usando “Model-Checking” com interpelantes e invariantes prove também que esta propriedade é um invariante em qualquer traço de $\,\Sigma\,$.


# Exercício 1 - Solução

In [ ]:
from pysmt.shortcuts import Symbol, Equals, GT, And, Or, Solver, Int, Not, Minus, Times, Plus, Div
from pysmt.typing import INT


def genState(vars,s,i):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i),INT)
    return state

def init(state, a, b, N):

    A = Equals(state['pc'], Int(0))
    B = GT(Int(a), Int(0))
    C = GT(Int(b), Int(0))
    D = GT(Int(N), Int(0))
    E = GT(Int(N), Int(a))
    F = GT(Int(N), Int(b))
    G = Equals(state['r'], Int(a))
    H = Equals(state['s'], Int(1))
    I = Equals(state['t'], Int(0))
    
    return And(A, B, C, D, E, F, G, H, I)

def error(state):
    return Equals(state['pc'],Int(2)) 

def trans(curr, prox, a, b):
           
    t01 = And(
        Equals(curr['pc'], Int(0)),
        Equals(prox['pc'], Int(1)),
        Equals(prox['r'], Int(a)),
        Equals(prox['r_prox'], Int(b)),
        Equals(prox['s'], Int(1)),
        Equals(prox['s_prox'], Int(0)),
        Equals(prox['t'], Int(0)),
        Equals(prox['t_prox'], Int(1)),
        Equals(curr['q'], prox['q'])
    )
    
    t14 = And(
        Equals(curr['pc'], Int(1)),
        Equals(prox['pc'], Int(4)),
        Equals(curr['r_prox'], Int(0)),
        Equals(curr['r'], prox['r']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prox'], prox['s_prox']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prox'], prox['t_prox']),
        Equals(curr['q'], prox['q'])
    )

    t12 = And(
        Equals(curr['pc'], Int(1)),
        Equals(prox['pc'], Int(2)),
        Not(Equals(curr['r_prox'], Int(0))),
        Equals(curr['r_prox'], prox['r_prox']),
        Equals(curr['r'], prox['r']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prox'], prox['s_prox']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prox'], prox['t_prox']),
        Equals(curr['q'], prox['q'])
    )
    
    division = Div(curr['r'], curr['r_prox'])
    
    t23 = And(
        Equals(curr['pc'], Int(2)),
        Equals(prox['pc'], Int(3)),
        Equals(prox['q'], division),
        Equals(curr['r'], prox['r']),
        Equals(curr['r_prox'], prox['r_prox']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prox'], prox['s_prox']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prox'], prox['t_prox'])
    )
    
    times_q_r_prox = Times(curr['q'], curr['r_prox'])
    times_q_s_prox = Times(curr['q'], curr['s_prox'])
    times_q_t_prox = Times(curr['q'], curr['t_prox'])
    
    minus_r_times = Minus(curr['r'], times_q_r_prox)
    minus_s_times = Minus(curr['s'], times_q_s_prox)
    minus_t_times = Minus(curr['t'], times_q_t_prox)

    t31 = And(
        Equals(curr['pc'], Int(3)),
        Equals(prox['pc'], Int(1)),
        Equals(prox['r'], curr['r_prox']),
        Equals(prox['r_prox'], minus_r_times),
        Equals(prox['s'], curr['s_prox']),
        Equals(prox['s_prox'], minus_s_times),
        Equals(prox['t'], curr['t_prox']),
        Equals(prox['t_prox'], minus_t_times),
        Equals(prox['q'], curr['q'])
    )

    t44 = And(
        Equals(curr['pc'], Int(4)),
        Equals(prox['pc'], Int(4)),
        Equals(curr['r'], prox['r']),
        Equals(curr['r_prox'], prox['r_prox']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prox'], prox['s_prox']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prox'], prox['t_prox']),
        Equals(curr['q'], prox['q'])
    )
    
    return Or(t01, t14, t12, t23, t31, t44)

def genTrace(vars,init,trans,error,n, a, b, N):
    with Solver(name="z3") as s:
        X = [genState(vars,'X',i) for i in range(n+1)]   # cria n+1 estados (com etiqueta X)
        I = init(X[0], a, b , N)
        Tks = [trans(X[i],X[i+1], a, b) for i in range(n)]
        
        if s.solve([I,And(Tks)]):      # testa se I /\ T^n  é satisfazível
            for i in range(n):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v]))

def kinduction_always(vars, init, trans, error, n, a, b, N, inv):
    with Solver(name="z3") as solver:
        X = [genState(vars, 'X', i) for i in range(n)]
        I = init(X[0], a, b, N)
        solver.add_assertion(I)
        
        for i in range(n-1):
            solver.add_assertion(trans(X[i], X[i+1], a, b))
        
        for i in range(n):
            solver.push()
            solver.add_assertion(Not(inv(X[i], a, b, N)))
            if solver.solve():
                print(f"> Contradição! O invariante não se verifica nos k estados iniciais.")
                for i, state in enumerate(X):
                    print(f"> State {i}: pc = {solver.get_value(state['pc'])}\nq = {solver.get_value(state['q'])}\ns = {solver.get_value(state['s'])}\nt = {solver.get_value(state['t'])}\nr = {solver.get_value(state['r'])}\ns' = {solver.get_value(state['s_prox'])}\nt' = {solver.get_value(state['t_prox'])}\nr' = {solver.get_value(state['r_prox'])}")
                return
            solver.pop()
        
        X2 = [declare(i+n) for i in range(n+1)]
        
        for i in range(n):
            solver.add_assertion(inv(X2[i], a, b, N))
            solver.add_assertion(trans(X2[i],X2[i+1], a, b))
        
        solver.add_assertion(Not(inv(X2[-1], a, b, N)))
        
        if solver.solve():
            print(f"> Contradição! O passo indutivo não se verifica.")
            for i, state in enumerate(X):
                print(f"> State {i}: pc = {solver.get_value(state['pc'])}\nq = {solver.get_value(state['q'])}\ns = {solver.get_value(state['s'])}\nt = {solver.get_value(state['t'])}\nr = {solver.get_value(state['r'])}\ns' = {solver.get_value(state['s_prox'])}\nt' = {solver.get_value(state['t_prox'])}\nr' = {solver.get_value(state['r_prox'])}")
            return
        
        print(f"> A propriedade verifica-se por k-indução (k={n}).")

def safety(state, a, b, N):
    A = GT(state['r'], Int(0))
    B = GT(Int(N), state['r'])
    C = Equals(state['r'], Plus(Times(Int(a), state['s']), Times(Int(b), state['t'])))
    return And(A, B, C)

In [ ]:
i = int(input('Insira o número do estado inicial: '))
K = int(input('Insira o número de estados: '))
N = int(input('Insira o maximizante de a e b: '))

In [ ]:
a = int(input('Insira o valor de a: '))
b = int(input('Insira o valor de b: '))

In [ ]:
vars = ["pc", "s", "t", "r", "s_prox", "t_prox", "r_prox", "q"]
genTrace(vars, init, trans, error, K, a, b, N)
kinduction_always(vars, init, trans, error, K, a, b, N, safety)